In [1]:
# get the client and server from resnet

from Models import resnet
weightpath = '/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/model/ResNet50.pth'
client, server = resnet.stupid_model_splitter(weight_path= weightpath, num_classes=10, device = 'cuda:0')

from Models import mobilenetv2
weightpath2 = '/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/model/MobileNetV2.pth'
client2, server2 = mobilenetv2.stupid_model_splitter(weight_path= weightpath2, num_classes=10, device = 'cuda:0')

from Dataloaders import dataloader_cifar10
train, test, classes = dataloader_cifar10.Dataloader_cifar10(train_batch=128, test_batch=100, seed=2024)

# get the result of 
client.eval()
client2.eval()
server.eval()
server2.eval()

client.to('cuda') 
client2.to('cuda')
server.to('cuda')
server2.to('cuda')

import torch
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(train):
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        outputs = client(inputs)
        outputs2 = client2(inputs)
        outputs3 = server(inputs)
        outputs4 = server2(inputs)
        if batch_idx == 0:
            break
    outputs3 = torch.max(outputs3, dim=1)[1]
    outputs4 = torch.max(outputs4, dim=1)[1]

    outputs3 = targets.eq(outputs3).sum().item()
    outputs4 = targets.eq(outputs4).sum().item()

    print('client1:', outputs)
    print('client2:', outputs2)

    outputs = torch.where(outputs == 0, 1, 0).sum().item()
    outputs2 = torch.where(outputs2 == 0, 1, 0).sum().item()

    print('client1:', outputs)
    print('client2:', outputs2)

    print('server1:', outputs3)
    print('server2:', outputs4)

    

RuntimeError: Error(s) in loading state_dict for resnet50_client:
	size mismatch for conv1.weight: copying a param with shape torch.Size([64, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 3, 7, 7]).

: 